In [ ]:
import numpy as np
import panel as pn

pn.extension()

In [ ]:
from PIL import Image

In [ ]:
what=np.array(Image.open(r"/home/matt/Downloads/orange.png"))

In [ ]:

%%time
from __future__ import annotations

import numpy as np

from bokeh import events
from bokeh.io import curdoc, show
from bokeh.layouts import column, row
from bokeh.models import Button, CustomJS, Div, TextInput
from bokeh.plotting import figure


def display_event(div: Div, attributes: list[str] = []) -> CustomJS:
    """
    Function to build a suitable CustomJS to display the current event
    in the div model.
    """
    style = 'float: left; clear: left; font-size: 13px'
    return CustomJS(args=dict(div=div), code=f"""
        const attrs = {attributes};
        const args = [];
        for (let i = 0; i < attrs.length; i++) {{
            const val = JSON.stringify(cb_obj[attrs[i]], function(key, val) {{
                return val.toFixed ? Number(val.toFixed(2)) : val;
            }})
            args.push(attrs[i] + '=' + val)
        }}
        const line = "<span style={style!r}><b>" + cb_obj.event_name + "</b>(" + args.join(", ") + ")</span>\\n";
        const text = div.text.concat(line);
        const lines = text.split("\\n")
        if (lines.length > 35)
            lines.shift();
        div.text = lines.join("\\n");
    """)



image = np.flipud(what.view("uint32").reshape(what.shape[:2]))


p = figure(tooltips=[("x", "$x"), ("y", "$y"), ("value", "@image")],tools=["box_select"])

p.image_rgba(image=[image],x=0, y=0, dw=what.shape[0], dh=what.shape[1])

# Add a div to display events and a button to trigger button click events

div = Div(width=1000)
layout = column(row(p, div))#,button, text_input)

# Register event callbacks

# Point events
point_attributes = ['x','y','sx','sy']
p.js_on_event(events.Tap,       display_event(div, attributes=point_attributes))
p.js_on_event(events.DoubleTap, display_event(div, attributes=point_attributes))
p.js_on_event(events.Press,     display_event(div, attributes=point_attributes))
p.js_on_event(events.PressUp,   display_event(div, attributes=point_attributes))

# Ranges Update events
p.js_on_event(events.RangesUpdate, display_event(div, attributes=['x0','x1','y0','y1']))

# Selection events
p.js_on_event(events.SelectionGeometry, display_event(div, attributes=['geometry', 'final']))

#curdoc().on_event(events.DocumentReady, display_event(div))
pn.panel(layout)

In [ ]:
image[1530,689]